In [12]:
import pandas as pd
intent = pd.read_csv('intent.csv')
persona = pd.read_csv('persona.csv')
subject = pd.read_csv('subject.csv')
template = pd.read_csv('template.csv')
pma = pd.read_csv('pma.csv')
match_type = {'match_type': ['br', 'xx']}
match_type = pd.DataFrame(match_type)
ad_group_name = {'ad_group': ['subject,intent,pma,match_type']}
ad_group_name = pd.DataFrame(ad_group_name)
location = pd.read_csv("location.csv")
market = pd.read_csv('market.csv', usecols=['language', 'market', 'account', 'account_id'])
campaign = pd.read_csv('campaign.csv', usecols=['market', 'campaign_name_template'])

In [13]:
intermediate = template.merge(pma, on='intent_localised_id', how='inner')[['intent_localised_id', 'intent_localised_x', 'keyword_template', 'language', 'pma']]

In [14]:
intermediate = intermediate.merge(match_type, how='cross')

In [15]:
intermediate = intermediate.merge(ad_group_name, how='cross')

In [16]:
intermediate = intermediate.merge(market, on='language', how='inner')

In [17]:
import numpy as np
from itertools import product

dimension_dfs = {
    'subject': subject,
    'persona': persona
}

def create_keywords(row):
    dimensions = row['keyword_template'].split(',')
    language = row['language']
    market = row['market']
    combinations = []

    for dimension in dimensions:

        if dimension == 'location':
            combinations.append(location['location'][location.market == market].values.tolist())

        if dimension == 'intent':
            combinations.append([row["intent_localised_x"]])
            continue
        
        if dimension in dimension_dfs:
            combinations.append(dimension_dfs[dimension].loc[dimension_dfs[dimension]['language'] == language, f'{dimension}_localised'].values.tolist())
        else:
            combinations.append([dimension])

    # Create all possible combinations
    combinations = list(product(*combinations))
    combinations = [' '.join(combination) for combination in combinations]

    return combinations

In [18]:
intermediate['keywords'] = intermediate.apply(create_keywords, axis=1)

In [19]:
intermediate = intermediate.explode('keywords')

In [22]:
intermediate = intermediate.merge(campaign, on='market', how='inner')

In [54]:
len(intermediate)

9975172

In [49]:
def retrieve_dimension(input_string, keyword_string, dimension): 
    
    split_string = input_string.split(",")

    subject_index = split_string.index(dimension)

    split_keyword = keyword_string.split(" ")

    retrieved_word = split_keyword[subject_index]
    
    return retrieved_word

In [50]:
def create_ad_group_name(row):
    subject = retrieve_dimension(row['keyword_template'], row['keywords'], "subject")
    intent = retrieve_dimension(row['keyword_template'], row['keywords'], "intent")
    pma = row['pma']
    match_type = row['match_type']

    ad_group_name = "_".join([subject, intent, pma, match_type])

    return ad_group_name

In [53]:
intermediate.apply(lambda x :create_ad_group_name(x), axis=1)

IndexError: list index out of range